In [ ]:
#Loads all imaging sequences for structural MRIs, and auto/manual annotations
def load_patient():
    # Import necessary module
    import slicer

    # Prompt the user for an integer
    patient_num = input("Enter the patient number (as an integer): ")

    # Pad the integer with leading zeros to total 5 digits
    patient_num_str = str(patient_num).zfill(5)

    # Create a folder for the patient in the Subject Hierarchy
    shNode = slicer.vtkMRMLSubjectHierarchyNode.GetSubjectHierarchyNode(slicer.mrmlScene)
    patient_folder_id = shNode.CreateFolderItem(shNode.GetSceneItemID(), f"UPENN-GBM-{patient_num_str}")

    # Define the file types
    file_types = ["T1", "T2", "T1GD", "Flair"]

    # Iterate over file types and load the corresponding volume
    for file_type in file_types:
        volume_path = f"C:\\Users\\Nicolas\\OneDrive - Rice University\\images_structural\\UPENN-GBM-{patient_num_str}_11\\UPENN-GBM-{patient_num_str}_11_{file_type}.nii.gz"
        try:
            volume = slicer.util.loadVolume(volume_path)
            if volume:
                # If the volume was loaded successfully, move it to the patient's folder
                shNode.SetItemParent(shNode.GetItemByDataNode(volume), patient_folder_id)
            else:
                print(f"Failed to load {file_type} volume from {volume_path}")
        except Exception:
            print(f"Could not load {file_type} volume for patient {patient_num_str}. Please check if the file exists and is in the correct format.")

    # Define the segmentation file paths
    auto_segm_path = f"C:\\Users\\Nicolas\\OneDrive - Rice University\\automated_segm\\UPENN-GBM-{patient_num_str}_11_automated_approx_segm.nii.gz"
    manual_segm_path = f"C:\\Users\\Nicolas\\OneDrive - Rice University\\images_segm\\UPENN-GBM-{patient_num_str}_11_segm.nii.gz"

    # Load the segmentation files into 3D Slicer
    try:
        auto_segm = slicer.util.loadSegmentation(auto_segm_path)
        if auto_segm:
            # If the segmentation was loaded successfully, move it to the patient's folder
            shNode.SetItemParent(shNode.GetItemByDataNode(auto_segm), patient_folder_id)
        else:
            print(f"Failed to load automated segmentation from {auto_segm_path}")
    except Exception:
        print(f"No automated segmentation available for patient {patient_num_str}.")

    try:
        manual_segm = slicer.util.loadSegmentation(manual_segm_path)
        if manual_segm:
            # If the segmentation was loaded successfully, move it to the patient's folder
            shNode.SetItemParent(shNode.GetItemByDataNode(manual_segm), patient_folder_id)
        else:
            print(f"Failed to load manual segmentation from {manual_segm_path}")
    except Exception:
        print(f"No manual segmentation available for patient {patient_num_str}.")

#Hides 3D visualizations
def hide_3d():
# Get the volume rendering node collection
    volume_rendering_nodes = slicer.mrmlScene.GetNodesByClass("vtkMRMLVolumeRenderingDisplayNode")

# Iterate through the collection and hide each volume rendering
    for i in range(volume_rendering_nodes.GetNumberOfItems()):
        volume_rendering_node = volume_rendering_nodes.GetItemAsObject(i)
        volume_rendering_node.SetVisibility(False)

# Get the segmentation node collection
    segmentation_nodes = slicer.mrmlScene.GetNodesByClass("vtkMRMLSegmentationNode")

# Iterate through the collection and hide each segmentation
    for i in range(segmentation_nodes.GetNumberOfItems()):
        segmentation_node = segmentation_nodes.GetItemAsObject(i)
        segmentation_display_node = segmentation_node.GetDisplayNode()
        segmentation_display_node.SetVisibility(False)


import slicer
from vtk import vtkPiecewiseFunction

#Renders selected tumor segements within structural MRI
def render_mri_segment(preset_name="MR-Default"):
    # Clear existing renders
    hide_3d()

    # Prompt the user for an integer
    patient_num = input("Enter the patient number (as an integer): ")

    # Pad the integer with leading zeros to total 5 digits
    patient_num_str = str(patient_num).zfill(5)

    # Prompt the user for the MRI type
    mri_type = input("Enter the MRI type (T1, T2, T1GD, Flair): ")

    # Define the names of the loaded volume and the automatic segmentation
    volume_name = f"UPENN-GBM-{patient_num_str}_11_{mri_type}"
    auto_segm_name = f"UPENN-GBM-{patient_num_str}_11_automated_approx_segm.nii.gz"

    # Get the volume node and the automatic segmentation data node
    volume_node = slicer.util.getNode(volume_name)
    auto_segm = slicer.util.getNode(auto_segm_name)

    # Check if the nodes were obtained
    if not volume_node:
        print(f"Failed to find the volume node {volume_name} in the scene.")
        return None
    if auto_segm is None:
        print(f"Could not obtain the data node for the automatic segmentation of patient {patient_num_str}.")
        return

    # Set the background volume of the slice views to the selected volume
    compositeNodes = slicer.mrmlScene.GetNodesByClass('vtkMRMLSliceCompositeNode')
    for i in range(compositeNodes.GetNumberOfItems()):
        compositeNode = compositeNodes.GetItemAsObject(i)
        compositeNode.SetBackgroundVolumeID(volume_node.GetID())

    # Enable slice intersection visibility for the selected volume
    auto_segm.GetDisplayNode().SetVisibility2D(True)

    # --- Volume rendering for the MRI data ---
    volumeRenderingLogic = slicer.modules.volumerendering.logic()
    display_node = volumeRenderingLogic.CreateDefaultVolumeRenderingNodes(volume_node)
    display_node.SetVisibility(True)
    volume_property_node = display_node.GetVolumePropertyNode()
    presets = slicer.modules.volumerendering.logic().GetPresetByName(preset_name)

    if presets is not None:
        volume_property_node.Copy(presets)
        # Set opacity threshold using transfer function
        transfer_function = vtkPiecewiseFunction()
        transfer_function.AddPoint(20, 0.0)
        transfer_function.AddPoint(70, 0.3)
        volume_property_node.SetScalarOpacity(transfer_function)
    else:
        print(f"Preset '{preset_name}' not found. Unable to apply preset.")

    # --- Displaying the segmentation data ---
    auto_segm.CreateClosedSurfaceRepresentation()

    # Initially set all segments to be invisible in 3D and 2D
    segmentation = auto_segm.GetSegmentation()
    for i in range(segmentation.GetNumberOfSegments()):
        segmentID = segmentation.GetNthSegmentID(i)
        auto_segm.GetDisplayNode().SetSegmentVisibility(segmentID, False)
        auto_segm.GetDisplayNode().SetSegmentVisibility3D(segmentID, False)

    # Ask user for each segment if they want to display it
    for i in range(segmentation.GetNumberOfSegments()):
        # Get each segment
        segmentID = segmentation.GetNthSegmentID(i)
        segmentName = segmentation.GetSegment(segmentID).GetName()

        # Prompt user if they want to display this segment
        display_choice = input(f"Do you want to display segment '{segmentName}' in 3D (Y/N)? ").strip().lower()

        # If user chose to display, show the segment in 3D and 2D slice views
        if display_choice == 'y':
            auto_segm.GetDisplayNode().SetSegmentVisibility(segmentID, True)
            auto_segm.GetDisplayNode().SetSegmentVisibility3D(segmentID, True)

    # Ensure that the segmentation is visible
    auto_segm.GetDisplayNode().SetVisibility(True)
    auto_segm.GetDisplayNode().SetVisibility3D(True)

    print(f"3D view and 2D slice windows for the automatic segmentation for patient {patient_num_str} set up according to your choices.")

    # Get layout manager
    layoutManager = slicer.app.layoutManager()

    # Get slice logic for each view and fit the slices
    for viewName in ['Red', 'Yellow', 'Green']:
        sliceLogic = layoutManager.sliceWidget(viewName).sliceLogic()
        sliceLogic.FitSliceToAll()

    # Reset 3D view to show all nodes
    threeDWidget = layoutManager.threeDWidget(0)
    threeDView = threeDWidget.threeDView()
    threeDView.resetFocalPoint()

    return display_node